In [1]:
import numpy as np
import pandas as pd
import loss_constraint as lc
import algorithm as al 
import matplotlib.pyplot as plt 
import os
from os import walk
import data_process as dp

In [5]:
'''
length, file_name, final_path = dp.path_process('D:/sun_python/PROJECT/生产调度/100组数据集/数据集')
left = [0,25,40,50,60,70,75,80,84,88,92,96]
right = [25,40,50,60,70,75,80,84,88,92,96,100]
L = []
FN = []
FP = []
RP = []
for i in range(0,12):
    L.append(length[left[i]:right[i]])
    FN.append(file_name[left[i]:right[i]])
    FP.append(final_path[left[i]:right[i]])
    RP.append(os.path.join(r'D:\sun_python\PROJECT\生产调度\figure\case_2','output_' + str(i+1) + '.xlsx'))
use_seed = False
#seed_path = r"D:\sun_python\PROJECT\生产调度\figure\case_1\out_put_check.xlsx"
seed_path = None
param_dict = {}
for i in range(0,12):
    param_dict[i] = (FP[i],RP[i],L[i],FN[i],use_seed,seed_path)
'''

In [6]:
'''
import datetime
import multiprocessing as mp
from tqdm import tqdm 
if __name__ == '__main__':

    start_t = datetime.datetime.now()

    num_cores = int(mp.cpu_count())
    print("本地计算机有: " + str(num_cores) + " 核心")
    pool = mp.Pool(12)
    
'''
    #param_dict = {'task1': (FP[0],RP[0],L[0],FN[0],True,seed_path),
    #              'task2': (FP[1],RP[1],L[1],FN[1],True,seed_path),
    #              'task3': (FP[2],RP[2],L[2],FN[2],True,seed_path),
    #              'task4': (FP[3],RP[3],L[3],FN[3],True,seed_path)}
'''
    results = [pool.apply_async(dp.process, args= param) for param in param_dict.values()]
    results = [p.get() for p in results]

    end_t = datetime.datetime.now()
    elapsed_sec = (end_t - start_t).total_seconds()
    print("多进程计算 共消耗: " + "{:.2f}".format(elapsed_sec) + " 秒")
'''

本地计算机有: 12 核心
多进程计算 共消耗: 179.32 秒


In [29]:
#将第一次的两个excel结果写到一起
sheet=pd.read_excel(r"D:\sun_python\PROJECT\生产调度\figure\case_1\output.xlsx",sheet_name=None)
#print(list(sheet.keys()))
sheet_res=pd.read_excel(r"D:\sun_python\PROJECT\生产调度\figure\case_1\output_res.xlsx",sheet_name=None)
#print(list(sheet_res.keys()))


In [31]:
writer = pd.ExcelWriter(r'D:\sun_python\PROJECT\生产调度\figure\case_1\out_put_check.xlsx')
for i in range(0,44):
    df = sheet[list(sheet.keys())[i]]
    col = []
    for j in range(1,df.shape[1]+1):
        col.append('Variable ' + str(j))
    df.columns = col
    df.to_excel(writer,list(sheet.keys())[i],index=False)

for i in range(44,100):
    df = sheet_res[list(sheet_res.keys())[i-44]]
    col = []
    for j in range(1,df.shape[1]+1):
        col.append('Variable ' + str(j))
    df.columns = col
    df.to_excel(writer,list(sheet_res.keys())[i-44],index=False)

writer.save()

C:\Users\Roy\AppData\Local\Temp\ipykernel_27744\836724476.py:18: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [2]:
length_set, file_name_set, final_path_set = dp.path_process('D:/sun_python/PROJECT/生产调度/100组数据集/数据集')
seed_parent_set=pd.read_excel(r"D:\sun_python\PROJECT\生产调度\figure\case_1\out_put_check.xlsx",sheet_name=None)
res_path = r'D:\sun_python\PROJECT\生产调度\figure\case_2'
use_seed = True
#seed_parent = None

params = []

for i in range(0,100):
    seed_parent = seed_parent_set[list(seed_parent_set.keys())[i]].to_numpy()
    length = length_set[i]
    file_name = file_name_set[i]
    data_path = final_path_set[i]
    params.append((data_path,res_path,length,file_name,use_seed,seed_parent))

In [3]:
from tqdm import tqdm
pbar = tqdm(total=100)
pbar.set_description('任务处理进程')
update = lambda *args: pbar.update()
import multiprocessing as mp
n_proc = 12
pool = mp.Pool(n_proc)
for param in params:
	pool.apply_async(dp.process, param, callback=update)
pool.close()
pool.join()


任务处理进程: 100%|██████████| 100/100 [21:40:11<00:00, 840.21s/it]   

In [13]:
path = r"D:\sun_python\PROJECT\生产调度\figure\case_2"
os.chdir(path)

# 遍历文件
res = []
for (dir_path, dir_name, file_name) in walk(path,topdown=True):
    res.extend(file_name)

# 获取excel文件地址
last_name = [i.split('.')[1] for i in res]
xlsx_res = []
for i in range(0,len(last_name)):
    if last_name[i] == 'xlsx':
        xlsx_res.append(res[i])
xlsx_res

length = np.array([eval(i.split('_')[1].split('.')[0]) for i in xlsx_res])
id_sort = np.argsort(length)
file_name_list = []
for i in id_sort:
    file_name_list.append(xlsx_res[i])

length = length[id_sort]

# 拼接全路径
final_path = []
for file_name in file_name_list:
    tmp_final_path = os.path.join(path, file_name)
    final_path.append(tmp_final_path)


In [17]:
writer = pd.ExcelWriter(r"D:\sun_python\PROJECT\生产调度\figure\case_2\case_2_result.xlsx")

for i in range(0,len(final_path)):
    path = final_path[i]
    df = pd.read_excel(path)

    col = []
    for j in range(1,df.shape[1]+1):
        col.append('Variable ' + str(j))
    df.columns = col
    df.to_excel(writer,file_name_list[i].split('.')[0],index=False)

writer.save()

C:\Users\Roy\AppData\Local\Temp\ipykernel_10276\1714546714.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
